# Embedding-based Machine Translation

Рассмотрим один из вариантов решения задачи машинного перевода, который не требует наличия параллельных корпусов.
В качестве языков для перевода будем использовать русский и украинский языки

### Каждая буква важна!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Список Сводеша

Список Сводеша (Swadesh list) — предложенный американским лингвистом Моррисом Сводешем инструмент для оценки степени родства между различными языками.


| Русский         | Белорусский              | Украинский              |
|-----------------|--------------------------|-------------------------|
| женщина         | жанчына, кабета, баба    | жінка                   |
| мужчина         | мужчына                  | чоловік, мужчина        |
| человек         | чалавек                  | людина, чоловік         |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            |
| жена            | жонка                    | дружина, жінка          |
| муж             | муж, гаспадар            | чоловiк, муж            |
| мать, мама      | маці, матка              | мати, матір, неня, мама |
| отец, тятя      | бацька, тата             | батько, тато, татусь    |
| много           | шмат, багата             | багато                  |
| несколько       | некалькі, колькі         | декілька, кілька        |
| другой, иной    | іншы                     | інший                   |
| зверь, животное | жывёла, звер, істота     | тварина, звір           |
| рыба            | рыба                     | риба                    |
| птица           | птушка                   | птах, птиця             |
| собака, пёс     | сабака                   | собака, пес             |
| вошь            | вош                      | воша                    |
| змея, гад       | змяя                     | змія, гад               |
| червь, червяк   | чарвяк                   | хробак, черв'як         |
| дерево          | дрэва                    | дерево                  |
| лес             | лес                      | ліс                     |
| палка           | кій, палка               | палиця                  |

## Embeddings

In [11]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Загружаем предобученные эмбеддинги для русского и украинского языков

In [12]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [13]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [14]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('август', 0.9999999403953552),
 ('июль', 0.9383155107498169),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729089498519897),
 ('декабрь', 0.8652556538581848),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [15]:
uk_emb.most_similar([uk_emb["серпень"]])

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('серпень', 0.9999998807907104),
 ('липень', 0.9096440076828003),
 ('вересень', 0.9016969799995422),
 ('червень', 0.8992518186569214),
 ('жовтень', 0.8810407519340515),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592806458473206),
 ('грудень', 0.8586862683296204),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256433606147766)]

In [16]:
ru_emb.most_similar([uk_emb["серпень"]])

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Недопустимость', 0.2443528175354004),
 ('конструктивность', 0.23293079435825348),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.2303171455860138),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608886659145355),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003723680973053),
 ('2015Александр', 0.21872761845588684),
 ('конструктивен', 0.21796570718288422)]

Загрузим словарь с соответствующими парами слов: **украинский -> русский**

In [17]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [18]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [19]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Сопоставление эмбеддингов (Embedding space mapping)

Пусть $x_i \in \mathrm{R}^d$ представление слова $i$ на исходном языке, $y_i \in \mathrm{R}^d$ представления на другом языке. Наша цель - обучить линейное преобразование $W$, которое минимизирует расстояние между $Wx_i$ и $y_i$ для некоторого набора слов.

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
или
$$W^*= \arg\min_W ||WX - Y||_F$$

где $||*||_F$ - норма Фробениуса.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ это просто линейная регрессия

In [23]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [24]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8541592359542847),
 ('июнь', 0.8411962389945984),
 ('март', 0.839739978313446),
 ('сентябрь', 0.8359214663505554),
 ('февраль', 0.8328748345375061),
 ('октябрь', 0.8311805129051208),
 ('ноябрь', 0.8278144598007202),
 ('июль', 0.8236349821090698),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8037999272346497)]

In [52]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        variants = [item[0] for item in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)]
        if ru in variants:
            num_matches += 1
    precision_val = num_matches / float(len(pairs))
    return precision_val


In [53]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [54]:
assert precision(uk_ru_test, X_test) == 0.0

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [55]:
assert precision(uk_ru_test, Y_test) == 1.0

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [56]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

print(precision_top1)
print(precision_top5)

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6356589147286822
0.8113695090439277


## Ортогональное преобразование

Попробуем найти ортогональное линейное преобразование

$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

$$I \text{- единичная матрица}$$

Найдем его с помощью сингулярного разложения
$$X^TY=U\Sigma V^T$$
$$W^*=UV^T$$

In [179]:
from sklearn.utils.extmath import randomized_svd

def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    u, sigma, vt = randomized_svd(np.matmul(X_train.transpose(), Y_train),
                                  n_components=300, n_iter=10, random_state=42)
    return np.matmul(u, vt)

In [180]:
W = learn_transform(X_train, Y_train)

In [181]:
W.shape

(300, 300)

In [94]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8237907290458679),
 ('сентябрь', 0.8049711585044861),
 ('март', 0.8025652766227722),
 ('июнь', 0.802183985710144),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934482097625732),
 ('февраль', 0.7914121150970459),
 ('июль', 0.790810763835907),
 ('август', 0.7891014814376831),
 ('декабрь', 0.7686370015144348)]

In [95]:
print(precision(uk_ru_test, np.matmul(X_test, W)))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.6537467700258398


In [96]:
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.8242894056847545


## Перевод текстов

Ниже представлены примеры переводов с украинского языка

In [126]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    result = ""
    words = sentence.split(" ")
    translated_words = []
    for word in words:
        translated_words.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0])
    return " ".join(translated_words)

In [128]:
print(translate("кіт зловив мишу"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


кот поймал мышку


In [129]:
print(translate("синій кіт"))

синий кот


/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [130]:
print(translate("синій кит"))

синий кит


/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [133]:
print(translate("Моя хата з краю , нічого не знаю"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Моя изба со края , ничего не знаю


In [134]:
print(translate("За дурною головою і ногам нема покою"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


За глупой головой и ногам нету покои


In [135]:
print(translate("Половина світу с жиру скаче — а половина с злиднів плаче"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


половина мира оло жира скачет — а половина оло нищеты плачет


In [137]:
print(translate("Сало без горілки , що свиня без рила"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Сало без водки , что свинья без рыла


In [149]:
print(translate("Чи бути , чи не бути — ось питання"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


? быть , ли не быть — вот вопрос


In [151]:
print(translate("Слава Україні"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Слава России


In [153]:
print(translate("Навіть боюся подумати що така квітка як Я , комусь дістанеться !"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Даже боюсь подумать что такая цветок как Я , кому-то достанется !


In [155]:
print(translate("У мене тільки два недоліки : я розумна і красива"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Во меня только два недостатки : мной умная и красивая


In [157]:
print(translate("Прапор тобі в руки , барабан на шию , сокиру в спину і електричку назустріч"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


флаг тебе во руки , барабан по шею , топор во спину и электричку навстречу


In [159]:
print(translate("Якщо ти плачеш не від щастя , то перестань …"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Если ты плачешь не от счастье , то перестань …


In [161]:
print(translate("Дівчата спочатку не думають , а потім думають , чому вони не думали , коли треба було думати ?"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


девчонки сперва не думают , а потом думают , почему они не думали , когда надо было думать ?


In [167]:
print(translate("Мужик сказав , мужик стукнув по столу , мужик з вікна речі ловить"))

/home/dolotov-e/tensorflow1.6_python3/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


мужик сказал , мужик стукнул по столу , мужик со окна вещи ловит
